In [ ]:
import os

import torch
from ipywidgets import interact
# from qsr_learning.data.data import draw, generate_objects

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Test `Entity` Class

In [ ]:
import math

from PIL import Image
from qsr_learning.entity import Entity

In [ ]:
canvas = Image.new("RGBA", (224, 224), (127, 127, 127, 127))
entity1 = Entity(
    name="octopus",
    absolute_direction=False,
    p=(30, 30),
    theta=0 / 360 * 2 * math.pi,
    size=(32, 32),
)
entity1.draw(canvas, show_bbox=True, orientation_marker=True)
entity2 = Entity(
    name="trophy",
    absolute_direction=False,
    p=(60, 60),
    theta=90 / 360 * 2 * math.pi,
    size=(32, 32),
)
entity2.draw(canvas, show_bbox=True, orientation_marker=True)

In [ ]:
entity1.image_array[:, :, 0].mean()

# Test Relations

In [ ]:
from ipywidgets import interact
from qsr_learning.relation import above, below, left_of, right_of


@interact(
    absolute_direction=(0, 1),
    x1=(0, 150),
    y1=(0, 150),
    theta1=(0, 360),
    x2=(0, 150),
    y2=(0, 150),
    theta2=(0, 360),
)
def test_spatial_relations(
    absolute_direction=1, x1=64, y1=64, theta1=0, x2=128, y2=128, theta2=150
):
    canvas = Image.new("RGBA", (224, 224), (127, 127, 127, 127))
    entity1 = Entity(
        name="octopus",
        absolute_direction=absolute_direction,
        p=(x1, y1),
        theta=theta1 / 360 * 2 * math.pi,
        size=(32, 32),
    )
    entity1.draw(canvas, show_bbox=True, orientation_marker=True)
    entity2 = Entity(
        name="trophy",
        absolute_direction=absolute_direction,
        p=(x2, y2),
        theta=theta2 / 360 * 2 * math.pi,
        size=(32, 32),
    )
    entity2.draw(canvas, show_bbox=True, orientation_marker=True)
    display(canvas)

    for relation in [left_of, right_of, above, below]:
        if relation(entity1, entity2):
            print(entity1.name, relation.__name__, entity2.name)
    for relation in [left_of, right_of, above, below]:
        if relation(entity2, entity1):
            print(entity2.name, relation.__name__, entity1.name)

# Test Negative Sample Generation

In [ ]:
import numpy as np
from munch import Munch
import random

from qsr_learning.data.data import generate_one_negative_example

In [ ]:
def test_negative_sample_generation():
    object_names = {"A", "B", "C"}
    relation_names = {"0", "1"}
    positive_examples = {("A", "0", "B")}

    mixture = Munch(head=1, relation=1, tail=1)
    negative_sample_types = list(mixture.keys())
    p = np.array(list(mixture.values()))
    p = p / p.sum()

    negative_examples = []

    for _ in range(10):
        negative_sample_type = np.random.choice(negative_sample_types, p=p)
        negative_example = generate_one_negative_example(
            object_names, relation_names, positive_examples, negative_sample_type
        )
        negative_examples.append(negative_example)

    return negative_examples

test_negative_sample_generation()

# Test a Trained Model

In [ ]:
def test_trained_model(model):
    def tensor2image(x):
        return Image.fromarray(
            (255 * ((0.5 * x) + 0.5)).numpy().astype("uint8").transpose(1, 2, 0)
        )

    def tensor2question(x):
        return [loader.train.dataset.idx2word[idx] for idx in x.numpy()]

    def tensor2answer(x):
        return x.item()

    for batch in loader.test:
        break

    images, questions, answers = (item.to(device) for item in batch)
    with torch.no_grad():
        model.eval()
        print(model(images[:1], questions[:1]) > 0.5)
        print(answers[:1])

    for i in range(batch[0].shape[0]):
        display(tensor2image(images[i].cpu()))
        display(tensor2question(questions[i].cpu()))
        display(tensor2answer(answers[i].cpu()))